In [30]:
import pandas as pd
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200/')

In [17]:
from time import time


In [4]:
train_data_path = "C:/Users/Aleksandra/PythonProjects/Loans/data/train.csv"
test_data_path = "C:/Users/Aleksandra/PythonProjects/Loans/data/test.csv"
train = pd.read_csv(train_data_path); print(train.shape)
test = pd.read_csv(test_data_path); print(test.shape)

(614, 13)
(367, 12)


In [63]:
CHUNKSIZE=100

index_name_train = "loan_prediction_train"
doc_type_train = "av-lp_train"

index_name_test = "loan_prediction_test"
doc_type_test = "av-lp_test"

In [99]:
def index_data(data_path, chunksize, index_name, doc_type,doc_id=None):
    f = open(data_path)
    csvfile = pd.read_csv(f, iterator=True, chunksize=chunksize) 
    try :
        es.indices.delete(index=index_name, ignore=[400, 404])
    except :
        pass
    res=es.indices.create(index=index_name, ignore=400)
    print(res)
    print("index created")
    for i,df in enumerate(csvfile): 
        print("i: ",i)
        records=df.where(pd.notnull(df), None).T.to_dict()
        list_records=[records[it] for it in records]
        
        try :
            print("indexing")
            #es.bulk_index(index_name, doc_type, list_records, id=i)
            #es.index(index=index_name, doc_type=doc_type, body=list_records, id=doc_id)
            #es.bulk(index = index_name, body = list_records)
            es.index(index=index_name,doc_type=doc_type,id=i,body=list_records)

            
        except :
            print("error!, skiping chunk!")
            pass

In [107]:
doc = {'Loan_ID': 'LP002949',
  'Gender': 'Female',
  'Married': 'No',
  'Dependents': '3+',
  'Education': 'Graduate',
  'Self_Employed': None,
  'ApplicantIncome': 416,
  'CoapplicantIncome': 41667,
  'LoanAmount': 350.0,
  'Loan_Amount_Term': 180,
  'Credit_History': None,
  'Property_Area': 'Urban',
  'Loan_Status': 'N'},

In [104]:
f = open(data_path)
csvfile = pd.read_csv(f, iterator=True, chunksize=chunksize) 
try :
    es.indices.delete(index=index_name, ignore=[400, 404])
except :
    pass
res=es.indices.create(index=index_name, ignore=400)
print(res)
print("index created")
for i,df in enumerate(csvfile): 
    print("i: ",i)
    records=df.where(pd.notnull(df), None).T.to_dict()
    list_records=[records[it] for it in records]

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'loan_prediction_train'}
index created
i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6


In [109]:
res = es.index(index=index_name, doc_type=doc_type, id=1, body=doc)
print(res)

PUT /loan_prediction_train/av-lp_train/1 [status:406 request:0.003s]


TransportError: TransportError(406, 'Content-Type header [] is not supported')

In [100]:
index_data(train_data_path, CHUNKSIZE, index_name_train, doc_type_train) # Indexing train data

PUT /loan_prediction_train/av-lp_train/0 [status:406 request:0.002s]
PUT /loan_prediction_train/av-lp_train/1 [status:406 request:0.002s]
PUT /loan_prediction_train/av-lp_train/2 [status:406 request:0.002s]
PUT /loan_prediction_train/av-lp_train/3 [status:406 request:0.002s]
PUT /loan_prediction_train/av-lp_train/4 [status:406 request:0.002s]


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'loan_prediction_train'}
index created
i:  0
indexing
error!, skiping chunk!
i:  1
indexing
error!, skiping chunk!
i:  2
indexing
error!, skiping chunk!
i:  3
indexing
error!, skiping chunk!
i:  4
indexing
error!, skiping chunk!
i:  5
indexing


PUT /loan_prediction_train/av-lp_train/5 [status:406 request:0.002s]
PUT /loan_prediction_train/av-lp_train/6 [status:406 request:0.002s]


error!, skiping chunk!
i:  6
indexing
error!, skiping chunk!


In [77]:
index_data(test_data_path, CHUNKSIZE, index_name_test, doc_type_test) # Indexing test data

POST /loan_prediction_test/av-lp_test [status:406 request:0.004s]
POST /loan_prediction_test/av-lp_test [status:406 request:0.004s]
POST /loan_prediction_test/av-lp_test [status:406 request:0.003s]


error!, skiping chunk!
error!, skiping chunk!
error!, skiping chunk!


POST /loan_prediction_test/av-lp_test [status:406 request:0.005s]


error!, skiping chunk!


In [74]:
es.get(index=index_name_train, doc_type=doc_type_train,id=1)

GET /loan_prediction_train/av-lp_train/1 [status:404 request:0.002s]


NotFoundError: TransportError(404, '{"_index":"loan_prediction_train","_type":"av-lp_train","_id":"1","found":false}')